In [28]:
import ctypes
import binascii
import typing
import math
import datetime
import enum
from types import SimpleNamespace
from pathlib import Path

import trio
import trio_serial
import anyio
import tqdm.notebook
import matplotlib.pyplot as plt
import serial.tools.list_ports
from IPython import display
import ipywidgets 
import bokeh.io
import bokeh.plotting
import bokeh.layouts
import bokeh as bk
import crcmod
import numpy as np
import scipy.ndimage
import pandas as pd
import attrs
import contextlib
import pint
import pint_pandas

from epymetrics.units import u, pandas_units

bk.io.output_notebook()
%matplotlib inline
%autoawait trio

Loading BokehJS ...

In [56]:
class Protocol(enum.IntEnum):
    hdr = 0x54
    data = 0x2c
    health = 0xe0
    mfg = 0x0f

ct = ctypes

class Point(ct.LittleEndianStructure):
    _pack_ = 1
    _fields_ = tuple(dict(
        distance = ct.c_uint16,
        intensity = ct.c_uint8,
    ).items())
    
class Data(ct.LittleEndianStructure):
    _pack_ = 1
    _fields_ = tuple(dict(
        speed = ct.c_uint16,
        start_angle = ct.c_uint16,
        points = Point*12,
        end_angle = ct.c_uint16,
        timestamp = ct.c_uint16,
    ).items())

    
class HealthData(ct.LittleEndianStructure):
    _pack_ = 1
    _fields_ = tuple(dict(
        error_code = ct.c_uint8,
    ).items())


class MfgData(ct.LittleEndianStructure):
    _pack_ = 1
    _fields_ = tuple(dict(
        speed = ct.c_uint16,
        product_version = ct.c_uint16,
        sn_high = ct.c_uint32,
        sn_low = ct.c_uint32,
        hardware_version = ct.c_uint32,
        software_version = ct.c_uint32,
    ).items())


packet_types = {
    Protocol.data: Data,
    Protocol.health: HealthData,
    Protocol.mfg: MfgData,
}

crcfun = crcmod.Crc(poly=0x14d,initCrc=0,xorOut=False,rev=False)
def calc_crc(data):
    return crcfun.new(data).crcValue

async def stream_points(ser):
    buf = b""
    while True:
        buf += await ser.receive_some()
        data = []
        while True:
            hdr = buf.find(Protocol.hdr)
            if hdr < 0:
                buf = b""
                break
            buf = buf[hdr:]
            if len(buf)<2:
                break
            try:
                cmd = Protocol(buf[1])
            except ValueError:
                # skip header of packet that we don't know and try again
                buf = buf[1:]
                continue
            ptype = packet_types.get(cmd)
            if ptype is None:
                # skip header of packet that we don't know and try again
                buf = buf[1:]
                continue
            sz = ctypes.sizeof(ptype)
            if len(buf)<sz+3:
                break
            expected = buf[sz+2]
            actual = calc_crc(buf[:sz+2])
            if actual != expected:
                print(f"CRC mismatch for packet {cmd}: expected 0x{expected:02x}, got 0x{actual:02x}")
                # skip header of corrupt packet, then try again
                buf = buf[1:]
                continue
            msg = ptype.from_buffer_copy(buf[2:sz+2])
            data.append(msg)
            buf = buf[sz+3:]
        if data:
            yield data
        

In [67]:
fig = bk.plotting.figure(match_aspect=True)

ds = bk.models.ColumnDataSource(data=dict(x=[],y=[],i=[]))

fig.scatter(x="x",y="y",size="i",source=ds)

hdl = bk.io.show(fig, notebook_handle=True)
alldata = []
tlp = trio.current_time()
async with trio_serial.SerialStream(port="/dev/cu.usbserial-0001",baudrate=230400) as ser:
    r,i,p = [],[],[]
    async for data in stream_points(ser):
        for d in data:
            sa = d.start_angle
            da = (d.end_angle - d.start_angle) % 36000
            ea = sa + da
            p.extend(np.linspace(sa,ea,len(d.points))*-np.pi/18000)
            i.extend(p.intensity for p in d.points)
            r.extend(p.distance for p in d.points)
        now = trio.current_time()
        if now < tlp+0.1:
            continue
        tlp = now
        r,i,p = (np.array(v) for v in (r,i,p))
        x,y = np.array([np.cos(p),np.sin(p)])*r
        ds.stream(dict(x=x,y=y,i=i*0.02),rollover=600)
        r,i,p = [],[],[]
        bk.io.push_notebook(handle=hdl)

CRC mismatch for packet 44: expected 0x1d, got 0xa5


KeyboardInterrupt: 

In [50]:
p.shape, r.shape, i.shape

((72,), (72,), (72,))